Customer KPIs (VIEW)

In [0]:
%sql
CREATE OR REPLACE VIEW sales_sv.vw_customer_kpis AS
SELECT
  c.customer_id,
  CONCAT(c.first_name,' ',c.last_name)                             AS full_name,
  MIN(c.signup_date)                                               AS signup_date,
  COUNT(o.order_id)                                                AS orders_count,
  SUM(CASE WHEN o.status = 'COMPLETED' THEN 1 ELSE 0 END)          AS completed_orders,
  SUM(CASE WHEN o.status = 'COMPLETED' THEN o.amount ELSE 0 END)   AS customer_spend,
  CASE WHEN COUNT(o.order_id) > 0
       THEN SUM(CASE WHEN o.status='COMPLETED' THEN o.amount ELSE 0 END) / NULLIF(COUNT(o.order_id),0)
       ELSE NULL END                                               AS aov
FROM workspace.sales_sv.customers_sv c
LEFT JOIN workspace.sales_sv.orders_sv o
  ON c.customer_id = o.customer_id
GROUP BY c.customer_id, full_name;

Product KPIs (VIEW)


In [0]:
%sql
CREATE OR REPLACE VIEW sales_sv.vw_product_kpis AS
SELECT
  p.product_id,
  p.product_name,
  p.category_id,
  SUM(oi.quantity)                                                 AS units_sold,
  SUM(CASE WHEN o.status='COMPLETED' THEN oi.line_amount ELSE 0 END) AS revenue,
  COUNT(DISTINCT CASE WHEN o.status='COMPLETED' THEN o.order_id END)  AS orders_count
FROM sales_sv.order_items_sv oi
JOIN common_sv.products_sv p  ON oi.product_id = p.product_id
JOIN sales_sv.orders_sv   o  ON oi.order_id   = o.order_id
GROUP BY p.product_id, p.product_name, p.category_id;

Order Status Distribution Insights for Sales Data

In [0]:
%sql
CREATE OR REPLACE VIEW sales_sv.vw_order_status_distribution AS
SELECT UPPER(status) AS status, COUNT(*) AS n_orders
FROM sales_sv.orders_sv
GROUP BY UPPER(status)
ORDER BY n_orders DESC;

Payment summary (VIEW)

In [0]:
%sql
CREATE OR REPLACE VIEW poc_sales.vw_payment_summary AS
SELECT
  payment_method,
  payment_status,
  COUNT(*)            AS tx_count,
  SUM(payment_amount) AS tx_amount
FROM sales_sv.payments_sv
GROUP BY payment_method, payment_status;

Shipping SLA (VIEW)

In [0]:
%sql
CREATE OR REPLACE VIEW poc_sales.vw_shipping_sla AS
SELECT
  s.order_id,
  DATE(o.order_ts)                                 AS order_date,
  DATE(s.ship_ts)                                  AS ship_date,
  DATEDIFF(DATE(s.ship_ts), DATE(o.order_ts))      AS days_to_ship,
  s.carrier, s.service
FROM sales_sv.shipments_sv s
JOIN sales_sv.orders_sv o ON s.order_id = o.order_id
WHERE o.status = 'COMPLETED';

One row per order, joined with customers, aggregated items, payments, shipments, refunds

In [0]:
%sql
CREATE OR REPLACE VIEW sales_gl.vw_universe_order AS
WITH
-- Items aggregated per order (join products & categories)
oi AS (
  SELECT
    oi.order_id,
    COUNT(*)                                     AS items_count,
    SUM(oi.quantity)                             AS items_qty_total,
    SUM(oi.line_amount)                          AS items_amount_total,
    COUNT(DISTINCT oi.product_id)                AS products_distinct,
    ARRAY_JOIN(COLLECT_SET(p.product_name), ', ')    AS products_csv,
    ARRAY_JOIN(COLLECT_SET(c.category_name), ', ')   AS categories_csv
  FROM sales_sv.order_items_sv oi
  JOIN sales_sv.products_sv p   ON oi.product_id = p.product_id
  JOIN sales_sv.categories_sv c ON p.category_id = c.category_id
  GROUP BY oi.order_id
),
-- Payments aggregated per order
pay AS (
  SELECT
    order_id,
    SUM(payment_amount)                                                 AS payment_amount_total,
    MAX(payment_ts)                                                   AS last_payment_date,
    ELEMENT_AT(SORT_ARRAY(COLLECT_LIST(payment_method)), 1)             AS any_payment_method,
    -- success flag if any settled payment
    MAX(CASE WHEN UPPER(payment_status) = 'SETTLED' THEN 1 ELSE 0 END)  AS has_settled_payment
  FROM sales_sv.payments_sv
  GROUP BY order_id
),
-- Shipments aggregated per order
ship AS (
  SELECT
    s.order_id,
    MIN(CAST(s.ship_ts AS DATE))                        AS ship_first_date,
    MAX(CAST(s.ship_ts AS DATE))                        AS ship_last_date,
    AVG(DATEDIFF(CAST(s.ship_ts AS DATE), CAST(o.order_ts AS DATE))) AS avg_days_to_ship,
    SUM(s.shipping_cost)                                 AS shipping_cost_total,
    ELEMENT_AT(SORT_ARRAY(COLLECT_LIST(s.carrier)), 1)   AS any_carrier,
    ELEMENT_AT(SORT_ARRAY(COLLECT_LIST(s.service)), 1)   AS any_service
  FROM sales_sv.shipments_sv s
  JOIN sales_sv.orders_sv    o ON s.order_id = o.order_id
  GROUP BY s.order_id
),
-- Refunds aggregated per order
ref AS (
  SELECT
    order_id,
    SUM(refund_amount) AS refund_amount_total,
    MAX(refund_ts)   AS last_refund_date
  FROM sales_sv.refunds_sv
  GROUP BY order_id
)
SELECT
  -- Order grain
  o.order_id,
  CAST(o.order_ts AS TIMESTAMP)                       AS order_ts,
  CAST(o.order_ts AS DATE)                            AS order_date,
  UPPER(o.status)                                     AS status,
  CAST(o.amount AS DECIMAL(18,2))                     AS order_amount,
  CASE WHEN UPPER(o.status) = 'COMPLETED' THEN o.amount ELSE 0 END AS revenue_net,
  -- Customer
  o.customer_id,
  CONCAT(c.first_name, ' ', c.last_name)              AS customer_name,
  LOWER(c.email)                                      AS customer_email,
  c.signup_date,
  -- Items (aggregated)
  COALESCE(oi.items_count, 0)                         AS items_count,
  COALESCE(oi.items_qty_total, 0)                     AS items_qty_total,
  COALESCE(oi.items_amount_total, 0)                  AS items_amount_total,
  COALESCE(oi.products_distinct, 0)                   AS products_distinct,
  COALESCE(oi.products_csv, '')                       AS products_csv,
  COALESCE(oi.categories_csv, '')                     AS categories_csv,
  -- Payments (aggregated)
  COALESCE(pay.payment_amount_total, 0)               AS payment_amount_total,
  pay.last_payment_date,
  pay.any_payment_method,
  COALESCE(pay.has_settled_payment, 0)                AS has_settled_payment,
  -- Shipments (aggregated)
  ship.ship_first_date,
  ship.ship_last_date,
  ship.avg_days_to_ship,
  COALESCE(ship.shipping_cost_total, 0)               AS shipping_cost_total,
  ship.any_carrier,
  ship.any_service,
  -- Refunds (aggregated)
  COALESCE(ref.refund_amount_total, 0)                AS refund_amount_total,
  ref.last_refund_date
FROM sales_sv.orders_sv o
LEFT JOIN sales_sv.customers_sv c ON o.customer_id = c.customer_id
LEFT JOIN oi   ON o.order_id = oi.order_id
LEFT JOIN pay  ON o.order_id = pay.order_id
LEFT JOIN ship ON o.order_id = ship.order_id
LEFT JOIN ref  ON o.order_id = ref.order_id;

One row per order_item, joined with product, category, order, customer, and order-level aggregates

In [0]:
%sql
CREATE OR REPLACE VIEW sales_gl.vw_universe_order_item AS
WITH
pay AS (
  SELECT order_id,
         SUM(payment_amount) AS payment_amount_total
  FROM sales_sv.payments_sv
  GROUP BY order_id
),
ship AS (
  SELECT
    s.order_id,
    AVG(DATEDIFF(CAST(s.ship_ts AS DATE), CAST(o.order_ts AS DATE))) AS avg_days_to_ship
  FROM sales_sv.shipments_sv s
  JOIN sales_sv.orders_sv    o ON s.order_id = o.order_id
  GROUP BY s.order_id
),
ref AS (
  SELECT order_id, SUM(refund_amount) AS refund_amount_total
  FROM sales_sv.refunds_sv
  GROUP BY order_id
)
SELECT
  -- Item grain
  oi.order_item_id,
  oi.order_id,
  oi.product_id,
  p.product_name,
  p.category_id,
  c.category_name,
  oi.quantity,
  oi.unit_price,
  oi.line_amount,
  -- Order context
  CAST(o.order_ts AS TIMESTAMP)         AS order_ts,
  CAST(o.order_ts AS DATE)              AS order_date,
  UPPER(o.status)                       AS order_status,
  CAST(o.amount AS DECIMAL(18,2))       AS order_amount,
  -- Customer context
  o.customer_id,
  CONCAT(cu.first_name, ' ', cu.last_name) AS customer_name,
  LOWER(cu.email)                         AS customer_email,
  -- Order-level aggregates
  COALESCE(pay.payment_amount_total,0)    AS payment_amount_total,
  COALESCE(ship.avg_days_to_ship, NULL)   AS avg_days_to_ship,
  COALESCE(ref.refund_amount_total,0)     AS refund_amount_total
FROM sales_sv.order_items_sv oi
JOIN sales_sv.products_sv  p   ON oi.product_id = p.product_id
JOIN sales_sv.categories_sv c  ON p.category_id = c.category_id
JOIN sales_sv.orders_sv    o   ON oi.order_id = o.order_id
LEFT JOIN sales_sv.customers_sv cu ON o.customer_id = cu.customer_id
LEFT JOIN pay  ON oi.order_id = pay.order_id
LEFT JOIN ship ON oi.order_id = ship.order_id
LEFT JOIN ref  ON oi.order_id = ref.order_id;